In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [3]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [4]:
# Example Selector

from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
example_selector = LengthBasedExampleSelector(
    examples=examples, example_prompt=example_prompt, max_length=100
)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
chain = prompt | chat
chain.invoke({"country": "Sweden"})

AI:
        Here is what I know:
        Capital: Stockholm
        Language: Swedish
        Food: Meatballs and herring
        Currency: Swedish Krona
        Famous for: IKEA, ABBA, and beautiful landscapes

AIMessageChunk(content='AI:\n        Here is what I know:\n        Capital: Stockholm\n        Language: Swedish\n        Food: Meatballs and herring\n        Currency: Swedish Krona\n        Famous for: IKEA, ABBA, and beautiful landscapes')

In [5]:
from langchain.prompts.example_selector.base import BaseExampleSelector


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_selector = RandomExampleSelector(examples=examples)
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)
chain = prompt | chat
chain.invoke({"country": "Sweden"})

AI:
        I know this:
        Capital: Stockholm
        Language: Swedish
        Food: Meatballs and herring
        Currency: Swedish Krona

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Stockholm\n        Language: Swedish\n        Food: Meatballs and herring\n        Currency: Swedish Krona')